In [28]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle
import scipy.stats.mstats as gmean
import matplotlib.pyplot as plt
import csv

if __name__ == "__main__":
    with open('../indice/ura_data_processed.csv', mode='r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
    dataset = pd.read_csv('../indice/ura_data_processed.csv', index_col=0)
    y = dataset['log_price_psf']

In [47]:
X_columns = [c for c in dataset.columns if not c.startswith('Period_') and not c in ['log_price_psf']]
X = dataset[X_columns]
X = sm.add_constant(X)
period_list = ['%sQ%s' % (year, qtr) for year in range(2016,2022) for qtr in range(1,5)]
period_list = period_list[:-3]
print(period_list)

['2016Q1', '2016Q2', '2016Q3', '2016Q4', '2017Q1', '2017Q2', '2017Q3', '2017Q4', '2018Q1', '2018Q2', '2018Q3', '2018Q4', '2019Q1', '2019Q2', '2019Q3', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1']


In [67]:
import numpy as np
 
# Calculating the exponential value of a scalar
exp_val = np.exp(3)
print(exp_val) # Output: 20.085536923187668
 
# Calculating the exponential value of an array
x = np.array([1, 2, 3])
exp_arr = np.exp(x)
print(exp_arr) # 

20.085536923187668
<class 'numpy.ndarray'>
[ 2.71828183  7.3890561  20.08553692]


In [90]:
import math
debug_predictions = {}
for period in period_list:
    with open('%s.pkl' % period, 'rb') as file:
        results = pickle.load(file)

    prediction = results.predict(X)
    prediction = prediction.astype(float)
    dataset['pred_psf_%s' % period] = np.exp(prediction)

pred_columns = [c for c in dataset.columns if c.startswith('pred_')]
base_period = '2016Q1'
base_period_filter = dataset['Period_%s' % base_period]
imputation_index_laspeyres = dataset.loc[base_period_filter,
                                         pred_columns].mean() / dataset[base_period_filter][pred_columns].mean()[0]

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_555/3433398503.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pred_columns].mean() / dataset[base_period_filter][pred_columns].mean()[0]


In [91]:
imputation_index_laspeyres.index = [x.replace('pred_psf_', '') for x in imputation_index_laspeyres.index]

In [92]:
district_9_filter = dataset['District_9'] == 1
imputation_index_9_laspeyres = dataset.loc[base_period_filter & 
                                           district_9_filter, pred_columns].mean() / dataset.loc[base_period_filter & 
                                                                                                 district_9_filter, pred_columns].mean()[0]
imputation_index_9_laspeyres.index = [x.replace('pred_psf_', '') for x in imputation_index_9_laspeyres.index]

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_555/3371077177.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  district_9_filter, pred_columns].mean() / dataset.loc[base_period_filter &


In [93]:
paasche_index_dict = {}
paasche_index_9_dict = {}
for period in period_list:
    period_filter = dataset['Period_%s' % period] == 1
    paasche_index_dict[period] = dataset.loc[period_filter]['pred_psf_%s' % period].mean() / dataset[period_filter]['pred_psf_%s' % base_period].mean()
    paasche_index_9_dict[period] = dataset.loc[period_filter & district_9_filter]['pred_psf_%s' % period].mean() / dataset[period_filter & district_9_filter]['pred_psf_%s' % base_period].mean()

In [96]:
##Imputation Index
imputation_index_paasche = pd.DataFrame.from_dict(paasche_index_dict, orient='index', columns = ['paasche_all'])

In [89]:
dataset

,log_price_psf,log_area_sqft,Type_Condominium,Building_age,Relative_tenure,Floor_number,District_1,District_2,District_4,District_5,...,pred_psf_2018Q4,pred_psf_2019Q1,pred_psf_2019Q2,pred_psf_2019Q3,pred_psf_2019Q4,pred_psf_2020Q1,pred_psf_2020Q2,pred_psf_2020Q3,pred_psf_2020Q4,pred_psf_2021Q1
S/N,,,,,,,,,,,,,,,,,,,,,
1,7.259820,7.366445,False,16.084873,0.899753,35.5,True,False,False,False,...,1713.962464,1749.808564,1760.679835,1731.826840,2037.431063,2047.188531,1979.849578,1968.297327,1819.346351,1833.911691
2,7.162397,7.512071,True,12.549723,1.000000,5.5,False,False,False,True,...,1166.323634,1172.997617,1208.496585,1230.883379,1190.986536,1219.213667,1206.491701,1204.475325,1190.010881,1329.213714
3,7.207860,7.228388,True,12.549723,1.000000,5.5,False,False,False,False,...,1491.083752,1447.644016,1534.654512,1602.000471,1500.600043,1490.358635,1497.366580,1564.655668,1569.717456,1672.617597
4,6.839476,7.563720,True,12.549723,1.000000,5.5,False,False,False,True,...,1157.473133,1165.189601,1201.453247,1223.800945,1183.846117,1212.218675,1199.029977,1196.102052,1181.814862,1322.562317
5,7.483244,6.535241,True,8.084873,0.916777,13.0,False,False,False,False,...,1470.019723,1430.907422,1466.291691,1521.971026,1467.707596,1443.636507,1439.530209,1524.687514,1518.830084,1585.488274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4663,6.894670,6.680855,True,2.083504,0.927387,5.5,False,False,False,False,...,1231.206585,1257.102681,1339.023257,1311.647292,1286.359934,1270.888976,1248.022414,1311.664518,1316.846579,1305.732010
4664,6.968850,6.818924,True,2.083504,0.927387,20.5,False,False,False,False,...,1259.406124,1324.429179,1407.247532,1364.564798,1338.754804,1329.047615,1285.503806,1352.504924,1370.032313,1347.426627
4665,7.198184,6.345636,False,12.549723,1.000000,5.5,False,False,False,False,...,1475.460085,1389.381872,1382.089960,1406.519647,1421.101393,1439.832283,1674.538504,1702.793171,1740.692051,1602.375736
